# Project

Welcome to the group project! The project is based on the [ACM RecSys 2021 Challenge](https://recsys-twitter.com/).

- Detailed information about the task, submission and grading can be found in a [dedicates site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1217340).
- Information about the dataset structure [on this site on TUWEL](https://tuwel.tuwien.ac.at/mod/page/view.php?id=1218810).

In [1]:
team_name = "team_5" # your team name e.g. 'team_1'
team_members = [("Simone Andreetto","01635069"),
                ("Adrian Bracher",""),
                ("Dominik Mailer","01634043"),
                ("Andreas Merckel",""),
                ("Dominik Pülke","12019262"),
                ("Sebastian Scholz","01526884"),
                ("Felix Winterleitner","01612776"),
                ("Ahmadou Wagne","12002293")] # [("Jane Doe","012345678"), ("John Doe","012345678")]


In [2]:
print(team_name)
print(team_members)

team_5
[('Simone Andreetto', '01635069'), ('Adrian Bracher', ''), ('Dominik Mailer', ''), ('Andreas Merckel', ''), ('Dominik Pülke', '12019262'), ('Sebastian Scholz', ''), ('Felix Winterleitner', '01612776'), ('Ahmadou Wagne', '12002293')]


In [3]:
path_to_data = '../shared/data/project/training/'
dataset_type = 'one_hour' # all_sorted, one_day, one_hour, one_week

In [19]:
import os
import re
import csv
import datetime

from model import reply_pred_model, retweet_pred_model, quote_pred_model, fav_pred_model 
from dataprep import import_data

In [24]:
# prepare models for U2U-Collaborative filtering
%run -i "u2u.py"
u2u_reply = Predictor("reply", imported_data)
u2u_retweet = Predictor("retweet", imported_data)
u2u_quote = Predictor("retweet_with_comment", imported_data)
u2u_fav = Predictor("like", imported_data)

In [16]:
print(imported_data)

                        engaging_user_id  like  reply  retweet  \
0       736278C2FEC488516CDA4ED6952A2154     0      0        0   
1       19D5367D835484236CAF9DBEF475FF7A     0      0        0   
2       40BEB04CF8D3CB02449879668656FFDB     0      0        0   
3       6415C94D3C27BA84C069DE049EBB3EDE     0      0        0   
4       7E614D5881BC18768880CC374C4BE821     0      0        0   
...                                  ...   ...    ...      ...   
891742  88A1D59A5ACB3CD87AFBCB58AFB33579     0      0        0   
891743  7D704E0ECA4D89F47460049DADEBBD65     0      0        0   
891744  58ED1F75F517E65F96C352C42C33E308     0      0        0   
891745  1B1C8C719C1BA40D5EE7DF756ACDF0AD     0      0        0   
891746  50E428669DE88261429DE5A10FED3DDE     0      0        0   

        retweet_with_comment                          tweet_id  
0                          1  395A05A1E8A0A4CEB2E623281C7A41EE  
1                          0  81E8247F4E74A0FCDBA911E1A3CB5412  
2           

In [5]:


all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

def parse_input_line(line):
    features = line #.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engaging_user_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    tweet_timestamp = features[all_features_to_idx['tweet_timestamp']]
    return tweet_id, user_id, input_feats, tweet_timestamp


def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id, user_id, features, tweet_timestamp = parse_input_line(row)   
                    reply_pred = reply_pred_model(features) # reply_model
                    retweet_pred = retweet_pred_model(features) # retweet_model
                    quote_pred = quote_pred_model(features) # pred_model
                    fav_pred = fav_pred_model(features) # fav_model
                    
                    # print(str(tweet_timestamp))
                    # print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')


In [28]:


all_features = ["text_tokens", "hashtags", "tweet_id", "present_media", "present_links", "present_domains",\
                "tweet_type","language", "tweet_timestamp", "engaged_with_user_id", "engaged_with_user_follower_count",\
               "engaged_with_user_following_count", "engaged_with_user_is_verified", "engaged_with_user_account_creation",\
               "engaging_user_id", "enaging_user_follower_count", "enaging_user_following_count", "enaging_user_is_verified",\
               "enaging_user_account_creation", "engagee_follows_engager"]

all_features_to_idx = dict(zip(all_features, range(len(all_features))))

def parse_input_line(line):
    features = line #.split("\x01")
    tweet_id = features[all_features_to_idx['tweet_id']]
    user_id = features[all_features_to_idx['engaging_user_id']]
    input_feats = features[all_features_to_idx['text_tokens']]
    tweet_timestamp = features[all_features_to_idx['tweet_timestamp']]
    return tweet_id, user_id, input_feats, tweet_timestamp


def evaluate_test_set():
    expanded_path = os.path.expanduser(path_to_data)
    part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
    part_files = sorted(part_files, key = lambda x:x[-5:]) 
        
    with open('results.csv', 'w') as output:
        for file in part_files:
            with open(file, 'r') as f:
                linereader = csv.reader(f, delimiter='\x01')
                last_timestamp = None
                for row in linereader:
                    tweet_id, user_id, features, tweet_timestamp = parse_input_line(row)   
                    reply_pred = u2u_reply.predict(user_id, tweet_id) # reply_model
                    retweet_pred = u2u_retweet.predict(user_id, tweet_id) # retweet_model
                    quote_pred = u2u_quote.predict(user_id, tweet_id) # pred_model
                    fav_pred = u2u_fav.predict(user_id, tweet_id) # fav_model
                    
                    # print(str(tweet_timestamp))
                    # print(str(reply_pred)+" "+str(retweet_pred)+" "+str(quote_pred)+" "+str(fav_pred))
                    
                    output.write(f'{tweet_id},{user_id},{reply_pred},{retweet_pred},{quote_pred},{fav_pred}\n')



In [29]:
expanded_path = os.path.expanduser(path_to_data)
part_files = [os.path.join(expanded_path, f) for f in os.listdir(expanded_path) if dataset_type in f]
part_files = sorted(part_files, key = lambda x:x[-5:]) 
part_files

['../shared/data/project/training/one_hour']

In [ ]:
evaluate_test_set()

In [ ]:

data = import_data(path_to_data+dataset_type)
data

In [ ]:
print(" -- unique tweets --")
print(data["tweet_id"].unique().size)
print(" -- engaged users --")
print(data["engaged_with_user_id"].unique().size)
print(" -- engaging users --")
print(data["engaging_user_id"].unique().size)

In [ ]:
# hidden


In [ ]:
from sklearn.metrics import average_precision_score, log_loss

def calculate_ctr(gt):
  positive = len([x for x in gt if x == 1])
  ctr = positive/float(len(gt))
  return ctr

def compute_rce(pred, gt):
    cross_entropy = log_loss(gt, pred)
    data_ctr = calculate_ctr(gt)
    strawman_cross_entropy = log_loss(gt, [data_ctr for _ in range(len(gt))])
    return (1.0 - cross_entropy/strawman_cross_entropy)*100.0

  
ground_truth = read_predictions(path_to_data + dataset_type) # will return data in the form (tweet_id, user_id, labed (1 or 0))
predictions = read_predictions("predictions.csv") # will return data in the form (tweet_id, user_id, prediction)

rce = compute_rce(predictions, ground_truth)
average_precision = average_precision_score(ground_truth, predictions)

print(rce)
print(average_precision)

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit

In [ ]:
# feel free to edit